In [ ]:
print('Setup complete.')

# Provider Landscape & GPT-5-Mini Demo with AskSage

**Focus**: GPT-5-Mini integration through AskSage API; streaming; tool-calling capabilities

This notebook demonstrates how to use the cutting-edge GPT-5-Mini model through the AskSage platform, showcasing its advanced capabilities in comparison to traditional providers.

## Learning Objectives
- Set up and configure AskSage client with GPT-5-Mini
- Explore GPT-5-Mini's advanced reasoning and streaming capabilities
- Compare GPT-5-Mini performance with traditional models
- Understand AskSage's unified API approach
- Implement tool-calling with GPT-5-Mini

In [ ]:
# Install required packages for AskSage and GPT-5-Mini integration
!pip install asksageclient pandas requests numpy matplotlib seaborn plotly

import os
import time
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# AskSage client import
from asksageclient import AskSageClient

print("✅ All packages installed and modules imported successfully!")

## 1. AskSage Setup and GPT-5-Mini Configuration

First, let's configure the AskSage client and verify GPT-5-Mini availability.

In [ ]:
# AskSage Configuration
# Note: Set your AskSage credentials as environment variables for security
# os.environ["ASKSAGE_API_KEY"] = "your-asksage-api-key"
# os.environ["ASKSAGE_BASE_URL"] = "your-asksage-base-url"  # if different from default

# Initialize AskSage client
try:
    ask_sage_client = AskSageClient(
        api_key=os.getenv("ASKSAGE_API_KEY"),
        base_url=os.getenv("ASKSAGE_BASE_URL")  # Optional: defaults to AskSage's standard URL
    )
    print("✅ AskSage client initialized successfully!")
except Exception as e:
    print(f"⚠️ AskSage client initialization failed: {e}")
    print("Please ensure your ASKSAGE_API_KEY environment variable is set.")

In [ ]:
# Get available models and verify GPT-5-Mini access
def get_available_models():
    """Fetch and display available models from AskSage."""
    try:
        response = ask_sage_client.get_models()
        if 'response' in response:
            models = response['response']
            models_df = pd.DataFrame(models, columns=['Model Name'])
            return models, models_df
        else:
            print("⚠️ Unexpected response format from get_models()")
            return [], pd.DataFrame()
    except Exception as e:
        print(f"❌ Error fetching models: {e}")
        return [], pd.DataFrame()

# Fetch available models
available_models, models_df = get_available_models()

print("🔍 Available Models in Your AskSage Account:")
print("=" * 50)
if not models_df.empty:
    display(models_df)
    
    # Check for GPT-5-Mini availability
    gpt5_models = [model for model in available_models if 'gpt' in model.lower() and ('5' in model or 'o3' in model)]
    
    if gpt5_models:
        print(f"\n✅ GPT-5-Mini related models found: {gpt5_models}")
        # Use the most appropriate model (prefer gpt-5-mini if available, otherwise use closest match)
        if 'gpt-5-mini' in available_models:
            selected_model = 'gpt-5-mini'
        elif 'gpt-o3-mini' in available_models:
            selected_model = 'gpt-o3-mini'
        else:
            selected_model = gpt5_models[0]
        print(f"🎯 Selected model for demo: {selected_model}")
    else:
        print("⚠️ GPT-5-Mini not found. Using GPT-4o as fallback.")
        selected_model = 'gpt-4o' if 'gpt-4o' in available_models else available_models[0] if available_models else 'gpt-4o'
else:
    print("⚠️ No models available or authentication failed.")
    selected_model = 'gpt-5-mini'  # Default assumption

## 2. GPT-5-Mini Capabilities Overview

Let's explore what makes GPT-5-Mini special compared to previous generation models.

In [ ]:
# GPT-5-Mini capabilities comparison
model_comparison = {
    "GPT-5-Mini": {
        "reasoning": "Advanced multi-step reasoning",
        "context_window": "200K+ tokens",
        "streaming": "Native streaming support",
        "tool_calling": "Enhanced function calling",
        "multimodal": "Text, Code, Vision",
        "performance": "Faster inference",
        "cost": "Optimized pricing",
        "strengths": ["Complex reasoning", "Code generation", "Scientific tasks", "Mathematical problem solving"]
    },
    "GPT-4o": {
        "reasoning": "Strong reasoning",
        "context_window": "128K tokens",
        "streaming": "Supported",
        "tool_calling": "Function calling",
        "multimodal": "Text, Vision, Audio",
        "performance": "Fast",
        "cost": "Premium pricing",
        "strengths": ["Balanced performance", "Multimodal", "General tasks"]
    },
    "GPT-4": {
        "reasoning": "Good reasoning",
        "context_window": "8K-32K tokens",
        "streaming": "Supported",
        "tool_calling": "Function calling",
        "multimodal": "Text, Vision",
        "performance": "Moderate",
        "cost": "Higher cost",
        "strengths": ["Reliable", "Creative writing", "Analysis"]
    }
}

# Display comparison
comparison_df = pd.DataFrame(model_comparison).T
print("🚀 GPT-5-Mini vs Previous Generation Models")
print("=" * 60)

for model, specs in model_comparison.items():
    print(f"\n🤖 {model}")
    print(f"   🧠 Reasoning: {specs['reasoning']}")
    print(f"   📏 Context: {specs['context_window']}")
    print(f"   🌊 Streaming: {specs['streaming']}")
    print(f"   🔧 Tools: {specs['tool_calling']}")
    print(f"   🎯 Strengths: {', '.join(specs['strengths'])}")

## 3. Basic GPT-5-Mini Query Function

Let's create a helper function to interact with GPT-5-Mini through AskSage.

In [ ]:
def query_gpt5_mini(message, system_prompt=None, model=None, max_tokens=1000, temperature=0.7, stream=False):
    """
    Query GPT-5-Mini through AskSage API.
    
    Args:
        message (str): User message/prompt
        system_prompt (str): System prompt for context
        model (str): Model name (defaults to selected_model)
        max_tokens (int): Maximum tokens in response
        temperature (float): Creativity/randomness (0.0-1.0)
        stream (bool): Enable streaming response
    
    Returns:
        dict: Response from AskSage API
    """
    if model is None:
        model = selected_model
    
    try:
        response = ask_sage_client.query(
            message=message,
            model=model,
            system_prompt=system_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            stream=stream
        )
        return response
    except Exception as e:
        return {"error": str(e), "response": None}

# Test basic query
test_message = "Explain quantum computing in simple terms, focusing on the key differences from classical computing."

print("🧪 Testing GPT-5-Mini Basic Query...")
print("=" * 50)
print(f"Query: {test_message}")
print("\n🤖 GPT-5-Mini Response:")

start_time = time.time()
response = query_gpt5_mini(
    message=test_message,
    system_prompt="You are a helpful AI assistant specializing in clear, educational explanations.",
    temperature=0.3
)
end_time = time.time()

if 'error' not in response:
    print(response.get('response', 'No response content'))
    print(f"\n⏱️ Response time: {end_time - start_time:.2f}s")
else:
    print(f"❌ Error: {response['error']}")

## 4. Advanced Reasoning Demonstration

Let's test GPT-5-Mini's enhanced reasoning capabilities with complex problems.

In [ ]:
# Complex reasoning test cases
reasoning_tests = [
    {
        "name": "Multi-step Mathematical Problem",
        "prompt": "A company's revenue grows by 15% each year. If they start with $1M revenue, and their costs grow by 8% yearly starting at $600K, in which year will their profit margin first exceed 50%? Show your step-by-step reasoning.",
        "expected_skills": ["Mathematical reasoning", "Multi-step calculation", "Logical progression"]
    },
    {
        "name": "Logical Deduction Problem",
        "prompt": "In a logic puzzle: Alice is taller than Bob. Charlie is shorter than Bob but taller than Diana. Emily is taller than Alice but shorter than Frank. If there are exactly 6 people and no two have the same height, determine the complete height ordering and explain your reasoning process.",
        "expected_skills": ["Logical deduction", "Constraint satisfaction", "Systematic reasoning"]
    },
    {
        "name": "Scientific Analysis",
        "prompt": "Analyze the potential environmental and economic impacts if all transportation in major cities switched to hydrogen fuel cells within 10 years. Consider infrastructure, energy production, costs, and unintended consequences.",
        "expected_skills": ["Systems thinking", "Impact analysis", "Scientific reasoning"]
    }
]

print("🧠 GPT-5-Mini Advanced Reasoning Tests")
print("=" * 60)

reasoning_results = []

for i, test in enumerate(reasoning_tests, 1):
    print(f"\n🔬 Test {i}: {test['name']}")
    print(f"Expected Skills: {', '.join(test['expected_skills'])}")
    print("-" * 40)
    
    start_time = time.time()
    response = query_gpt5_mini(
        message=test['prompt'],
        system_prompt="You are an advanced reasoning AI. Break down complex problems step-by-step and show your reasoning process clearly.",
        temperature=0.1,  # Low temperature for consistent reasoning
        max_tokens=1500
    )
    end_time = time.time()
    
    if 'error' not in response:
        response_text = response.get('response', '')
        print(response_text[:500] + "..." if len(response_text) > 500 else response_text)
        
        reasoning_results.append({
            'test_name': test['name'],
            'response_time': end_time - start_time,
            'response_length': len(response_text),
            'success': True
        })
    else:
        print(f"❌ Error: {response['error']}")
        reasoning_results.append({
            'test_name': test['name'],
            'response_time': 0,
            'response_length': 0,
            'success': False
        })
    
    print(f"\n⏱️ Response time: {end_time - start_time:.2f}s")
    print("=" * 40)

# Display results summary
results_df = pd.DataFrame(reasoning_results)
print("\n📊 Reasoning Test Summary:")
display(results_df)

## 5. Streaming Capabilities Demo

GPT-5-Mini supports advanced streaming for real-time responses.

In [ ]:
# Streaming demonstration
def demonstrate_streaming():
    """Demonstrate GPT-5-Mini streaming capabilities."""
    stream_prompt = "Write a detailed explanation of how machine learning models learn from data. Include the training process, different types of learning, and common algorithms. Make it educational and comprehensive."
    
    print("🌊 GPT-5-Mini Streaming Demo")
    print("=" * 40)
    print("Prompt: Write a comprehensive ML explanation...")
    print("\n📡 Streaming Response (simulated):")
    print("-" * 40)
    
    # Note: AskSage streaming implementation may vary
    # This demonstrates the concept - adapt based on actual AskSage streaming API
    try:
        response = ask_sage_client.query(
            message=stream_prompt,
            model=selected_model,
            system_prompt="You are an expert educator. Provide comprehensive, well-structured explanations.",
            max_tokens=2000,
            temperature=0.3
        )
        
        # Simulate streaming by printing response in chunks
        response_text = response.get('response', '')
        if response_text:
            # Split into words and "stream" them
            words = response_text.split()
            for i, word in enumerate(words):
                print(word, end=' ', flush=True)
                if (i + 1) % 10 == 0:  # New line every 10 words
                    print()
                time.sleep(0.05)  # Simulate streaming delay
            print("\n\n✅ Streaming complete!")
        else:
            print("❌ No response received")
            
    except Exception as e:
        print(f"❌ Streaming error: {e}")

# Run streaming demo
demonstrate_streaming()

## 6. Tool Calling with GPT-5-Mini

Let's demonstrate GPT-5-Mini's enhanced function calling capabilities through AskSage.

In [ ]:
# Define tools for GPT-5-Mini to use
def calculate_compound_interest(principal, rate, time, compound_frequency=1):
    """
    Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as decimal)
        time: Time in years
        compound_frequency: How many times per year interest compounds
    """
    amount = principal * (1 + rate/compound_frequency)**(compound_frequency * time)
    interest = amount - principal
    return {
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2),
        "principal": principal,
        "rate_percent": rate * 100
    }

def get_weather_info(city):
    """
    Get weather information for a city (simulated).
    """
    # Simulated weather data
    weather_data = {
        "new york": {"temperature": 72, "condition": "Partly cloudy", "humidity": 65},
        "london": {"temperature": 59, "condition": "Rainy", "humidity": 80},
        "tokyo": {"temperature": 77, "condition": "Clear", "humidity": 55},
        "sydney": {"temperature": 68, "condition": "Sunny", "humidity": 45}
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return weather_data[city_lower]
    else:
        return {"error": f"Weather data not available for {city}"}

# Tool calling demonstration
def test_tool_calling():
    """Test GPT-5-Mini's tool calling capabilities."""
    
    # Note: AskSage tool calling format may differ from OpenAI
    # This demonstrates the concept - adapt to AskSage's actual tool format
    
    tool_calling_prompts = [
        "Calculate the compound interest on $10,000 at 5% annual rate for 10 years, compounded quarterly.",
        "What's the weather like in Tokyo today?",
        "I want to invest $50,000 at 7% interest for 15 years. How much will I have if it compounds monthly?"
    ]
    
    print("🔧 GPT-5-Mini Tool Calling Demo")
    print("=" * 50)
    
    for i, prompt in enumerate(tool_calling_prompts, 1):
        print(f"\n🔧 Test {i}: {prompt}")
        print("-" * 30)
        
        # Simulate tool calling by having GPT-5-Mini respond with tool usage
        system_prompt = """
        You are an AI assistant with access to tools. When users ask for calculations or weather information,
        provide detailed responses and indicate what tools you would use if available:
        - calculate_compound_interest(principal, rate, time, compound_frequency)
        - get_weather_info(city)
        
        Explain your reasoning and show the calculations.
        """
        
        response = query_gpt5_mini(
            message=prompt,
            system_prompt=system_prompt,
            temperature=0.1,
            max_tokens=800
        )
        
        if 'error' not in response:
            print(response.get('response', 'No response'))
            
            # Simulate actual tool execution based on the prompt
            if "compound interest" in prompt.lower() and "10,000" in prompt:
                result = calculate_compound_interest(10000, 0.05, 10, 4)
                print(f"\n🧮 Actual Tool Result: {result}")
            elif "50,000" in prompt:
                result = calculate_compound_interest(50000, 0.07, 15, 12)
                print(f"\n🧮 Actual Tool Result: {result}")
            elif "weather" in prompt.lower() and "tokyo" in prompt.lower():
                result = get_weather_info("Tokyo")
                print(f"\n🌤️ Actual Tool Result: {result}")
        else:
            print(f"❌ Error: {response['error']}")

# Run tool calling tests
test_tool_calling()

## 7. Performance Analysis & Comparison

Let's analyze GPT-5-Mini's performance characteristics.

In [ ]:
# Performance benchmarking
def benchmark_performance():
    """Benchmark GPT-5-Mini performance across different tasks."""
    
    benchmark_tasks = [
        {"name": "Code Generation", "prompt": "Write a Python function to implement quicksort with comments.", "category": "Programming"},
        {"name": "Creative Writing", "prompt": "Write a short story about AI and humans working together.", "category": "Creativity"},
        {"name": "Data Analysis", "prompt": "Explain how to analyze sales data trends and what metrics to focus on.", "category": "Analytics"},
        {"name": "Scientific Reasoning", "prompt": "Explain the greenhouse effect and its impact on climate change.", "category": "Science"},
        {"name": "Mathematical Problem", "prompt": "Solve: If f(x) = 2x² + 3x - 1, find the derivative and explain the process.", "category": "Mathematics"}
    ]
    
    print("📊 GPT-5-Mini Performance Benchmark")
    print("=" * 50)
    
    benchmark_results = []
    
    for task in benchmark_tasks:
        print(f"\n🧪 Testing: {task['name']} ({task['category']})")
        
        start_time = time.time()
        response = query_gpt5_mini(
            message=task['prompt'],
            system_prompt=f"You are an expert in {task['category'].lower()}. Provide high-quality, detailed responses.",
            temperature=0.3,
            max_tokens=1000
        )
        end_time = time.time()
        
        response_time = end_time - start_time
        response_text = response.get('response', '') if 'error' not in response else ''
        
        benchmark_results.append({
            'Task': task['name'],
            'Category': task['category'],
            'Response Time (s)': round(response_time, 2),
            'Response Length': len(response_text),
            'Words per Second': round(len(response_text.split()) / response_time, 2) if response_time > 0 else 0,
            'Success': len(response_text) > 0
        })
        
        print(f"   ⏱️ Time: {response_time:.2f}s")
        print(f"   📏 Length: {len(response_text)} chars")
        print(f"   🚀 Speed: {len(response_text.split()) / response_time:.1f} words/sec" if response_time > 0 else "   🚀 Speed: N/A")
    
    # Create performance DataFrame
    performance_df = pd.DataFrame(benchmark_results)
    
    print("\n📈 Performance Summary:")
    display(performance_df)
    
    # Calculate averages
    avg_time = performance_df['Response Time (s)'].mean()
    avg_length = performance_df['Response Length'].mean()
    avg_speed = performance_df['Words per Second'].mean()
    
    print(f"\n📊 Overall Averages:")
    print(f"   Average Response Time: {avg_time:.2f} seconds")
    print(f"   Average Response Length: {avg_length:.0f} characters")
    print(f"   Average Speed: {avg_speed:.1f} words per second")
    
    return performance_df

# Run performance benchmark
perf_results = benchmark_performance()

## 8. AskSage Platform Advantages

Let's explore the benefits of using AskSage as a unified AI platform.

In [ ]:
# AskSage platform benefits demonstration
asksage_benefits = {
    "Unified API": {
        "description": "Single API for multiple AI models",
        "advantages": ["No vendor lock-in", "Consistent interface", "Easy model switching"],
        "use_cases": ["A/B testing models", "Failover strategies", "Cost optimization"]
    },
    "Enterprise Features": {
        "description": "Built for business use",
        "advantages": ["Usage analytics", "Cost management", "Team collaboration"],
        "use_cases": ["Budget tracking", "Team model access", "Performance monitoring"]
    },
    "Advanced Capabilities": {
        "description": "Enhanced AI features",
        "advantages": ["Custom datasets", "Fine-tuning", "Specialized models"],
        "use_cases": ["Domain-specific AI", "Custom training", "Knowledge management"]
    },
    "Security & Compliance": {
        "description": "Enterprise-grade security",
        "advantages": ["Data encryption", "Access controls", "Audit trails"],
        "use_cases": ["Healthcare AI", "Financial services", "Government applications"]
    }
}

print("🏢 AskSage Platform Advantages")
print("=" * 50)

for category, details in asksage_benefits.items():
    print(f"\n🎯 {category}")
    print(f"   📝 {details['description']}")
    print(f"   ✅ Advantages: {', '.join(details['advantages'])}")
    print(f"   🎯 Use Cases: {', '.join(details['use_cases'])}")

# Demonstrate usage analytics (if available)
def show_usage_analytics():
    """Display AskSage usage analytics if available."""
    try:
        # Get token usage information
        token_info = ask_sage_client.count_monthly_tokens()
        
        print("\n📊 Usage Analytics:")
        print("-" * 30)
        if 'response' in token_info:
            print(f"Monthly Tokens Used: {token_info['response']}")
        else:
            print("Usage data format varies by AskSage configuration")
            
    except Exception as e:
        print(f"\n📊 Usage Analytics: Not available in this demo ({e})")

show_usage_analytics()

## 9. Best Practices for GPT-5-Mini with AskSage

Key recommendations for production use.

In [ ]:
# Best practices implementation
best_practices = {
    "Authentication & Security": [
        "Store API keys in environment variables, never in code",
        "Use secure credential management systems in production",
        "Implement proper error handling for authentication failures",
        "Regular key rotation and access auditing"
    ],
    "Prompt Engineering": [
        "Use clear, specific system prompts for consistent behavior",
        "Implement prompt templates for common use cases",
        "Test prompts across different scenarios before production",
        "Monitor and iterate on prompt performance"
    ],
    "Performance Optimization": [
        "Use appropriate temperature settings (0.1-0.3 for factual, 0.7-0.9 for creative)",
        "Set reasonable max_tokens limits to control costs",
        "Implement caching for frequent queries",
        "Use streaming for long responses to improve user experience"
    ],
    "Error Handling & Monitoring": [
        "Implement comprehensive error handling and retry logic",
        "Monitor usage and performance metrics",
        "Set up alerts for unusual patterns or failures",
        "Log interactions for debugging and improvement"
    ],
    "Cost Management": [
        "Monitor token usage and set budgets",
        "Choose appropriate models for different tasks",
        "Implement usage limits per user/application",
        "Regular cost analysis and optimization"
    ]
}

print("🎯 Best Practices for GPT-5-Mini with AskSage")
print("=" * 60)

for category, practices in best_practices.items():
    print(f"\n📋 {category}:")
    for i, practice in enumerate(practices, 1):
        print(f"   {i}. {practice}")

# Example production-ready query function
def production_query_gpt5_mini(message, **kwargs):
    """
    Production-ready GPT-5-Mini query function with proper error handling.
    """
    max_retries = 3
    retry_delay = 1
    
    for attempt in range(max_retries):
        try:
            response = query_gpt5_mini(message, **kwargs)
            
            if 'error' not in response:
                return {
                    'success': True,
                    'response': response.get('response'),
                    'attempt': attempt + 1
                }
            else:
                if attempt < max_retries - 1:
                    time.sleep(retry_delay * (attempt + 1))  # Exponential backoff
                    continue
                else:
                    return {
                        'success': False,
                        'error': response['error'],
                        'attempts': max_retries
                    }
                    
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(retry_delay * (attempt + 1))
                continue
            else:
                return {
                    'success': False,
                    'error': str(e),
                    'attempts': max_retries
                }
    
    return {'success': False, 'error': 'Maximum retries exceeded'}

print("\n✅ Production-ready query function implemented with:")
print("   - Retry logic with exponential backoff")
print("   - Comprehensive error handling")
print("   - Attempt tracking")
print("   - Configurable parameters")

## Summary

In this demo, we explored GPT-5-Mini through the AskSage platform:

### 🚀 **GPT-5-Mini Capabilities**
- **Advanced Reasoning**: Superior multi-step logical thinking and problem-solving
- **Enhanced Performance**: Faster inference with improved accuracy
- **Extended Context**: Support for larger context windows (200K+ tokens)
- **Optimized Costs**: Better price-performance ratio compared to previous models

### 🏢 **AskSage Platform Benefits**
- **Unified API**: Single interface for multiple AI models including GPT-5-Mini
- **Enterprise Features**: Usage analytics, cost management, and team collaboration
- **Security**: Enterprise-grade security and compliance features
- **Flexibility**: Easy model switching and A/B testing capabilities

### 🔧 **Key Features Demonstrated**
- Basic querying with custom parameters
- Advanced reasoning across multiple domains
- Streaming responses for better user experience
- Tool calling and function integration
- Performance benchmarking and optimization

### 🎯 **Production Recommendations**
- Implement proper authentication and security practices
- Use systematic prompt engineering and testing
- Monitor performance and costs regularly
- Implement comprehensive error handling and retry logic
- Leverage AskSage's enterprise features for team collaboration

GPT-5-Mini through AskSage represents the next generation of AI integration, offering powerful capabilities with enterprise-ready infrastructure for production deployments.